In [1]:
import glob
from io import open
#import bibtexparser
#from bibtexparser.bparser import BibTexParser
import pandas as pd
from pandas import DataFrame as df
from os import path

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## Merge CSV databases

### Reading the column names

In [31]:
labels = pd.read_csv("data/PsycInfo/csv/Acronym Key.csv")

In [33]:
column_to_name = dict(labels.values)

In [35]:
dfs = []
for file in glob.glob("data/PsycInfo/csv/*csv"):
  word = path.basename(file).split('.')[0].split('_')[0]
  if word == "Acronym Key": continue
  df_ = pd.read_csv(file, encoding="iso-8859-1", header=1)
  df_.insert(0, "Term", [word]*len(df_))
  print(file, len(df_))
  dfs.append(df_)

data/PsycInfo/csv/multiculturalism_3.csv 1000
data/PsycInfo/csv/multiculturalism_6.csv 1624
data/PsycInfo/csv/multicultural_5.csv 1000
data/PsycInfo/csv/interracial_2.csv 1246
data/PsycInfo/csv/multiculturalism_2.csv 1000
data/PsycInfo/csv/multicultural_9.csv 1000
data/PsycInfo/csv/polyethnic.csv 7
data/PsycInfo/csv/multiculturalism_5.csv 1000
data/PsycInfo/csv/bicultural.csv 1784
data/PsycInfo/csv/multiculturalism_1.csv 1000
data/PsycInfo/csv/multicultural_10.csv 1000
data/PsycInfo/csv/polyculturalism.csv 13
data/PsycInfo/csv/biethnic.csv 42
data/PsycInfo/csv/other race.csv 738
data/PsycInfo/csv/other ethnicity.csv 87
data/PsycInfo/csv/mixed race.csv 551
data/PsycInfo/csv/interracial_1.csv 1500
data/PsycInfo/csv/polycultural.csv 14
data/PsycInfo/csv/multicultural_3.csv 1000
data/PsycInfo/csv/biracial.csv 882
data/PsycInfo/csv/multicultural_4.csv 1000
data/PsycInfo/csv/multiculturalism_4.csv 1000
data/PsycInfo/csv/mixed ethnicity.csv 99
data/PsycInfo/csv/multicultural_2.csv 1000
data/P

In [44]:
words_df = pd.concat(dfs)
assert len(words_df[words_df.Term == 'biracial']) == 882
words_df.rename(columns=column_to_name).head()

,AB,AI,Accession Number,Author(s),Type of Book,Corporate/Institutional Author,PsycINFO Classification Code,Conference,Chapter Author(s),Copyright,...,Supplemental Material,Chapter Title,Test DOI,Title,Tests & Measures,Term,Update Code,Vendor,Volume,Date
0,(from the chapter) This chapter provides Infor...,NaN,Book: 2011-12790-011.,"Abakoui, Roki\n\nSimmons, Rosemary E",Handbook/Manual,NaN,Professional Psychological & Health Personnel ...,NaN,NaN,NaN,...,Web Sites [Internet Available],NaN,NaN,Sizeism: An unrecognized prejudice. [References].,NaN,multiculturalism,20120618 (PsycINFO),Ovid Technologies,NaN,2010
1,(from the chapter) The purpose of this chapter...,NaN,Book: 2011-12790-010.,"Schreier, Barry A\n\nDudley Lassiter, Kim",Handbook/Manual,NaN,Professional Psychological & Health Personnel ...,NaN,NaN,NaN,...,NaN,NaN,NaN,Competencies for working with sexual orientati...,NaN,multiculturalism,20120618 (PsycINFO),Ovid Technologies,NaN,2010
2,(from the chapter) This chapter provides an ov...,NaN,Book: 2011-12790-009.,"Sanchez, Delida\n\nDavis, Claytie III",Handbook/Manual,NaN,Professional Psychological & Health Personnel ...,NaN,NaN,NaN,...,NaN,NaN,NaN,Becoming a racially competent therapist. [Refe...,NaN,multiculturalism,20120618 (PsycINFO),Ovid Technologies,NaN,2010
3,(from the chapter) There is a growing number o...,NaN,Book: 2011-12790-008.,"Dixon-Peters, Aisha",Handbook/Manual,NaN,Professional Psychological & Health Personnel ...,NaN,NaN,NaN,...,Web Sites [Internet Available],NaN,NaN,Developing multicultural competency in clinica...,NaN,multiculturalism,20120618 (PsycINFO),Ovid Technologies,NaN,2010
4,(from the chapter) The examination of the diff...,NaN,Book: 2011-12790-007.,"Stevens, Mark A\n\nEnglar-Carlson, Matt",Handbook/Manual,NaN,Professional Psychological & Health Personnel ...,NaN,NaN,NaN,...,Web Sites [Internet Available],NaN,NaN,Psychotherapy with men: Building practice comp...,NaN,multiculturalism,20120618 (PsycINFO),Ovid Technologies,NaN,2010


In [45]:
words_df.to_csv("data/PsycInfo/processed/psychinfo_combined.csv", encoding='utf-8')

## Load PsychINFO unified database

In [ ]:
psychinfo = df.from_csv("psychinfo_combined.csv", encoding='utf-8')

In [ ]:
_ = psychinfo[['word', 'year']].hist(figsize=(18,12), by='word')